# Collecting some baselines from various models

In [150]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

%matplotlib inline

#### Display metrics using show_results()

In [151]:
def show_results(test_type, accuracy, cross_val, conf_mat):
    print()
    print(test_type)
    print('Accuracy:\t\t\t', accuracy)
    print('Cross Validation Results:\t', cross_val)
    print('Cross Validation Mean:\t\t', np.mean(cross_val))
    print('Confusion Matrix:\n', conf_mat)

### Load csv into dataframe 

In [152]:
df = pd.read_csv('Autism_Data.csv')
df.columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 
              'A7', 'A8', 'A9', 'A10', 'age', 'gender',
             'ethnicity', 'jundice', 'autism', 'country', 
             'app', 'result', 'age_category', 'relation',
             'app_prediction']
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,gender,ethnicity,jundice,autism,country,app,result,age_category,relation,app_prediction
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,United States,no,6,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2,18 and more,?,NO


## Convert yes/no colums to 1/0 columns

In [153]:
gender_row = pd.Series(np.where(df['gender'] == 'm', 1, 0))
jund_row = pd.Series(np.where(df['jundice'] == 'yes', 1, 0))
autism_row = pd.Series(np.where(df['autism'] == 'yes', 1, 0))

df['gender'] = gender_row
df['jundice'] = jund_row
df['autism'] = autism_row

df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,gender,ethnicity,jundice,autism,country,app,result,age_category,relation,app_prediction
0,1,1,1,1,0,0,1,1,0,0,...,0,White-European,0,0,United States,no,6,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,1,Latino,0,1,Brazil,no,5,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,1,Latino,1,1,Spain,no,8,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,0,White-European,0,1,United States,no,6,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,0,?,0,0,Egypt,no,2,18 and more,?,NO


## Establish X, y datasets.  
### Set global paramaters for models found below. 

In [154]:
X = df[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'gender', 'jundice']]
y = df['autism']

test_size = 0.3
random_state = 1
alpha = 1e-5
cv = 5

# solver = {sgd, adam, lbfgs}
solver = 'lbfgs'

# hidden_layer_sizes = tuple, ith element is number of neurons in ith layer.
hidden_layer_sizes = (20, 14)

orig_text = 'Original data'
smote_text = 'Preprocessed with SMOTE'

# Dicts to keep track of scores
orig_dict = {}
smote_dict = {}

## Multi Layer Perceptron Classifier (original data)

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

clf = MLPClassifier(solver=solver, 
                    alpha=alpha, 
                    hidden_layer_sizes=hidden_layer_sizes, 
                    random_state=1)

clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
orig_dict['MLP'] = accuracy
cross_val = cross_val_score(clf, X, y, cv=cv)
conf_mat = confusion_matrix(y_test, clf.predict(X_test))

show_results(orig_text, accuracy, cross_val, conf_mat)


Original data
Accuracy:			 0.8443396226415094
Cross Validation Results:	 [0.80985915 0.82269504 0.78014184 0.80714286 0.85      ]
Cross Validation Mean:		 0.8139677783010116
Confusion Matrix:
 [[174  20]
 [ 13   5]]


## Multi Layer Perceptron Classifier (data after SMOTE)

In [156]:
X_smote, y_smote = SMOTE().fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=101)

clf = MLPClassifier(solver=solver, 
                    alpha=alpha, 
                    hidden_layer_sizes=hidden_layer_sizes, 
                    random_state=1)

clf.fit(X_train, y_train)

smote_accuracy = clf.score(X_test, y_test)
smote_dict['MLP'] = smote_accuracy
cross_val = cross_val_score(clf, X_smote, y_smote, cv=5)
conf_mat = confusion_matrix(y_test, clf.predict(X_test))

show_results(smote_text, accuracy, cross_val, conf_mat)


Preprocessed with SMOTE
Accuracy:			 0.8443396226415094
Cross Validation Results:	 [0.80487805 0.88211382 0.89430894 0.89344262 0.8852459 ]
Cross Validation Mean:		 0.8719978675196588
Confusion Matrix:
 [[162  37]
 [ 12 157]]


## Support Vector Machine (Original data)

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

clf_svm = svm.SVC()
clf_svm.fit(X_train, y_train)

svm_accuracy = clf_svm.score(X_test, y_test)
orig_dict['SVM'] = svm_accuracy
cross_val_svm = cross_val_score(clf_svm, X, y)
conf_mat_svm = confusion_matrix(y_test, clf_svm.predict(X_test))

show_results(orig_text, svm_accuracy, cross_val_svm, conf_mat_svm)


Original data
Accuracy:			 0.9150943396226415
Cross Validation Results:	 [0.86864407 0.87179487 0.87179487]
Cross Validation Mean:		 0.8707446037954513
Confusion Matrix:
 [[194   0]
 [ 18   0]]


## Support Vector Machine (with SMOTE)

In [158]:
X_smote, y_smote = SMOTE().fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=101)

clf_svm_smote = svm.SVC()
clf_svm_smote.fit(X_train, y_train)

svm_accuracy = clf_svm_smote.score(X_test, y_test)
smote_dict['SVM'] = svm_accuracy
cross_val_svm = cross_val_score(clf_svm_smote, X, y)
conf_mat_svm = confusion_matrix(y_test, clf_svm_smote.predict(X_test))

show_results(smote_text, svm_accuracy, cross_val_svm, conf_mat_svm)


Preprocessed with SMOTE
Accuracy:			 0.7119565217391305
Cross Validation Results:	 [0.86864407 0.87179487 0.87179487]
Cross Validation Mean:		 0.8707446037954513
Confusion Matrix:
 [[126  73]
 [ 33 136]]


## KNN (original data)

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

knn_accuracy = knn.score(X_test, y_test)
orig_dict['KNN'] = knn_accuracy
cross_val_knn = cross_val_score(knn, X, y)
conf_mat_knn = confusion_matrix(y_test, knn.predict(X_test))

show_results(orig_text, knn_accuracy, cross_val_knn, conf_mat_knn)


Original data
Accuracy:			 0.8820754716981132
Cross Validation Results:	 [0.83474576 0.85042735 0.85042735]
Cross Validation Mean:		 0.8452001545221884
Confusion Matrix:
 [[186   8]
 [ 17   1]]


## KNN (with SMOTE)

In [160]:
X_smote, y_smote = SMOTE().fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=101)

knn_smote = KNeighborsClassifier(n_neighbors=3)
knn_smote.fit(X_train, y_train)

knn_accuracy = knn_smote.score(X_test, y_test)
smote_dict['KNN'] = knn_accuracy
cross_val_knn = cross_val_score(knn_smote, X_smote, y_smote)
conf_mat_knn = confusion_matrix(y_test, knn_smote.predict(X_test))

show_results(smote_text, knn_accuracy, cross_val_knn, conf_mat_knn)


Preprocessed with SMOTE
Accuracy:			 0.8016304347826086
Cross Validation Results:	 [0.75121951 0.80882353 0.84313725]
Cross Validation Mean:		 0.8010600988362825
Confusion Matrix:
 [[134  65]
 [  8 161]]


## Guassian Process Classifier (original data)

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

gpc = GaussianProcessClassifier()
gpc.fit(X_train, y_train)

gpc_accuracy = gpc.score(X_test, y_test)
orig_dict['GPC'] = gpc_accuracy
cross_val_gpc = cross_val_score(gpc, X, y)
conf_mat_gpc = confusion_matrix(y_test, gpc.predict(X_test))

show_results(orig_text, gpc_accuracy, cross_val_gpc, conf_mat_gpc)


Original data
Accuracy:			 0.9150943396226415
Cross Validation Results:	 [0.86864407 0.87179487 0.87179487]
Cross Validation Mean:		 0.8707446037954513
Confusion Matrix:
 [[194   0]
 [ 18   0]]


## Guassian Process Classifier (with SMOTE)

In [162]:
X_smote, y_smote = SMOTE().fit_sample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3, random_state=101)

gpc_smote = GaussianProcessClassifier()
gpc_smote.fit(X_train, y_train)

gpc_accuracy = gpc_smote.score(X_test, y_test)
smote_dict['GPC'] = gpc_accuracy
cross_val_gpc = cross_val_score(gpc_smote, X_smote, y_smote)
conf_mat_gpc = confusion_matrix(y_test, gpc_smote.predict(X_test))

show_results(orig_text, gpc_accuracy, cross_val_gpc, conf_mat_gpc)


Original data
Accuracy:			 0.7527173913043478
Cross Validation Results:	 [0.7804878  0.80147059 0.78186275]
Cross Validation Mean:		 0.787940379403794
Confusion Matrix:
 [[133  66]
 [ 25 144]]


## Comparing results

In [163]:
print('without smote:')
for key, val in orig_dict.items():
    print(key, ':  ', val)

print('\n\nwith smote:')
for key, val in smote_dict.items():
    print(key, ':  ', val)


without smote:
MLP :   0.8443396226415094
SVM :   0.9150943396226415
KNN :   0.8820754716981132
GPC :   0.9150943396226415


with smote:
MLP :   0.8668478260869565
SVM :   0.7119565217391305
KNN :   0.8016304347826086
GPC :   0.7527173913043478
